In [1]:
import os
import json
import numpy as np
import pandas as pd
import scipy.sparse as spr

목차

- 1. 데이터 처리 (불러오기,추가,수정,변환)
- 2. 플레이리스트로 추천 (협업필터링)

# 데이터 처리

## train data 불러오기

In [2]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)
song_df = pd.DataFrame.from_dict(song_dict)

## train_df 전처리

### 일반 전처리
- num_songs,num_tags = '중복제거한 노래 태그 개수'

In [3]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


In [4]:
train_df['tags_cnt'] = train_df['tags'].map(lambda x : len(x))
train_df['songs_cnt'] = train_df['songs'].map(lambda x : len(x))

In [5]:
def list_merger(x):
    temp_list.extend(x)

temp_list = []

train_df['songs'].map(list_merger)

unique_songs = list(set(temp_list))

num_songs = len(unique_songs)

In [6]:
temp_list = []
                
train_df['tags'].map(list_merger)
                    
unique_tags = list(set(temp_list))
                
num_tags = len(unique_tags)

In [7]:
num_songs

615142

In [8]:
num_tags

29160

In [9]:
num_train = len(train_df)
num_train

115071

### tag에 id부여

In [10]:
train_df

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,10,38
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,53
...,...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,7,12
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,1,11
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,8,11
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,6,55


In [11]:
# { 태그 : 새로운id } 딕셔너리
tag_to_id = {}


# { 새로운id : 태그 } 딕셔너리
id_to_tag = {}

# enumerate 반복 횟수를 이용해서 새로운 id 부여
for i,v in enumerate(unique_tags):
    id_to_tag[i] = v

# id_to_tag의 키 밸류를 반전해서 tag_to_id를 만듦
tag_to_id = {v:k for k,v in id_to_tag.items()}


# tags를 id로 변환해서 새로운 column을 생성
def tag_id_trans(x):
    
    temp_list = []
    
    for tag in x:
        temp_list.append(tag_to_id[tag])
    
    return temp_list

train_df['tags_id'] = train_df['tags'].map(tag_id_trans)

In [12]:
train_df.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,tags_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[16883]
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42,"[20374, 25245]"
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28,"[1799, 13526]"
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,10,38,"[23975, 22188, 14450, 4941, 1540, 15131, 3618,..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,53,[16526]


### 플레이리스트 속 노래들(songs)에게 새로운 id 부여

In [13]:
# { 노래id : 새로운id } 딕셔너리
song_to_id = {}


# { 새로운id : 노래id } 딕셔너리
id_to_song = {}

# enumerate 반복 횟수를 이용해서 새로운 id 부여
for i,v in enumerate(unique_songs):
    id_to_song[i] = v

# id_to_tag의 키 밸류를 반전해서 tag_to_id를 만듦
song_to_id = {v:k for k,v in id_to_song.items()}


# tags를 id로 변환해서 새로운 column을 생성
def song_id_trans(x):
    
    temp_list = []
    
    for song in x:
        temp_list.append(song_to_id[song])
    
    return temp_list

train_df['new_songs_id'] = train_df['songs'].map(song_id_trans)

In [14]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,tags_id,new_songs_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[16883],"[456704, 112732, 333158, 488440, 258853, 12127..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42,"[20374, 25245]","[375894, 587314, 431997, 104605, 338568, 21226..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28,"[1799, 13526]","[72132, 240434, 144495, 161861, 307991, 222934..."


### 플레이리스트 새로운 id 부여

In [15]:
train_df['new_plyst_id'] = train_df.index

In [16]:
id_to_playlist = {}

playlist_to_id = {}

def mk_dict(x):
    
    id_to_playlist[x['new_plyst_id']] = x['id']

train_df.loc[:,['id','new_plyst_id']].apply(mk_dict,axis=1)

playlist_to_id = {v:k for k,v in id_to_playlist.items()}

In [17]:
train_df = train_df[['tags','tags_id','id','new_plyst_id','plylst_title','songs','new_songs_id','tags_cnt','songs_cnt','like_cnt','updt_date']]
train_df.head(3)

,tags,tags_id,id,new_plyst_id,plylst_title,songs,new_songs_id,tags_cnt,songs_cnt,like_cnt,updt_date
0,[락],[16883],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,"[추억, 회상]","[20374, 25245]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]","[1799, 13526]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


In [18]:
train_df.columns = ['태그','태그_id','플리_id','새플리_id','플리제목','노래_id','새노래_id','태그수','노래수','좋아요수','갱신일']
train_df.head(3)

,태그,태그_id,플리_id,새플리_id,플리제목,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일
0,[락],[16883],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,"[추억, 회상]","[20374, 25245]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]","[1799, 13526]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


### 변수 정리

<현재>

- num_songs : 중복 제거된 총 노래수 615142
- num_tags : 중복 제거된 총 태그수 29160
- num_train : 데이터의 총 갯수 115071
 
- tag_to_id : { 태그 : 새로운id } 딕셔너리
- id_to_tag : { 새로운id : 태그 } 딕셔너리
- song_to_id : { 노래 : 새로운id } 딕셔너리
- id_to_song : { 새로운id : 노래 } 딕셔너리
- playlist_to_id { 플리id : 새로운id } 딕셔너리 
- id_to_playlist { 새로운id : 플리id } 딕셔너리 

# 태그로 노래를 추천해보자

In [19]:
# 중복을 제거한 tag 리스트

tag_all = []

def collect_tag(x):
    global tag_all
    
    tag_all.extend(x)
    
train_df['태그_id'].map(collect_tag)

tag_unique = list(set(tag_all))

In [20]:
# 각 태그가 몇번 출몰 했는지 세어 놓은 딕셔너리

from collections import Counter

tag_count = dict(Counter(tag_all))

tag_count = {i:tag_count[i] for i in range(len(tag_count))}

In [21]:
sum(list(tag_count.values()))

476331

In [22]:
# 태그_id에 달린 장르id들을 넣을 딕셔너리 초기화
tid_sid_dict = {i:[] for i in range(len(tag_unique))}

In [23]:
train_df.head(3)

,태그,태그_id,플리_id,새플리_id,플리제목,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일
0,[락],[16883],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,"[추억, 회상]","[20374, 25245]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]","[1799, 13526]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


In [24]:
# 태그_id에 달린 장르id들을 list로 넣어주기

def tagid_songid(x):
    
    global tid_sid_dict
    temp_list = []
    for i in x['태그_id']:
        tid_sid_dict[i].extend(x['새노래_id'])

train_df[['태그_id','새노래_id']].apply(tagid_songid,axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
115066    None
115067    None
115068    None
115069    None
115070    None
Length: 115071, dtype: object

In [25]:
tid_sid_dict

{0: [220026,
  234826,
  262064,
  118208,
  207078,
  11554,
  192493,
  586718,
  411994,
  376303,
  486625,
  70230,
  58475,
  492274,
  570059,
  394917,
  369503,
  518847,
  69087,
  515645,
  610977,
  160509,
  65680,
  320134,
  309378],
 1: [202340,
  115607,
  343725,
  216,
  254817,
  565145,
  177755,
  352592,
  542620,
  505514,
  600071,
  83615,
  204070,
  395577,
  505514,
  34631,
  509675,
  40002,
  67843,
  479323,
  316063,
  5764,
  440003,
  397308,
  544658,
  46123,
  449264,
  151145,
  609550,
  41877,
  366344,
  272474,
  509760,
  250229,
  601615,
  535969,
  90417,
  21668,
  106341,
  565383,
  163837,
  183603,
  565784,
  410023,
  216,
  254817,
  565145,
  319161,
  1875,
  419322,
  118136,
  414064,
  17622,
  178336,
  242420,
  499795,
  19598,
  315433,
  353071,
  130224,
  20044,
  504914,
  134678,
  302456,
  416484],
 2: [450705,
  341948,
  336295,
  246672,
  165869,
  164278,
  319888,
  532254,
  86233,
  150335,
  429252,
  3693

In [26]:
for i in range(len(tid_sid_dict)):
    v_count = Counter(tid_sid_dict[i])
    v_count_sorted = sorted(v_count.items(),key=(lambda x:x[1]),reverse=True)

    temp_list = []
    
    if len(v_count_sorted) > 4:
        for k,v in v_count_sorted[:5]:
            temp_list.append((k,v))
    else:
        for k,v in v_count_sorted:
            temp_list.append((k,v))
            
    tid_sid_dict[i] = temp_list

    
tid_sid_dict

{0: [(220026, 1), (234826, 1), (262064, 1), (118208, 1), (207078, 1)],
 1: [(216, 2), (254817, 2), (565145, 2), (505514, 2), (202340, 1)],
 2: [(450705, 1), (341948, 1), (336295, 1), (246672, 1), (165869, 1)],
 3: [(82799, 1), (497256, 1), (396936, 1), (17475, 1), (500381, 1)],
 4: [(157434, 1), (299073, 1), (89830, 1), (449956, 1), (367696, 1)],
 5: [(538050, 1), (74581, 1), (303610, 1), (288840, 1), (594040, 1)],
 6: [(25620, 1), (313539, 1), (479555, 1), (199285, 1), (286452, 1)],
 7: [(97650, 3), (267574, 3), (225154, 3), (568857, 3), (283464, 3)],
 8: [(23622, 1), (217210, 1), (532931, 1), (362368, 1), (360675, 1)],
 9: [(322253, 2), (210847, 2), (91829, 2), (590654, 2), (400734, 2)],
 10: [(334031, 1), (2645, 1), (9351, 1), (490845, 1), (384139, 1)],
 11: [(192030, 1), (442108, 1), (130339, 1), (199217, 1), (551487, 1)],
 12: [(317547, 2), (112809, 2), (225719, 2), (100987, 2), (283399, 1)],
 13: [(80375, 1), (66728, 1), (224209, 1), (317433, 1), (287877, 1)],
 14: [(384263, 1), 

In [27]:
# 포함된 노래의 개수를 가지고 있는 list
song_len = [len(v) for v in tid_sid_dict.values()]

In [28]:
#  태그별 장르포함 횟수만큼 반복  * 2번 태그는 7번 나왔으니 2를 7번 찍기
row = np.repeat(range(len(tid_sid_dict)),song_len)
len(row)

145706

In [29]:
col =  [song[0] for k,v in tid_sid_dict.items() for song in v]
len(col)

145706

In [30]:
# 태그 마다 달린 노래의 총길이 만큼 1찍기
dat = np.repeat(1, sum(song_len))
len(dat)

145706

In [31]:
train_tags_A = spr.csr_matrix((dat, (row, col)), shape=(train_df['태그수'].sum(), len(song_to_id)))

train_tags_A_T = train_tags_A.T.tocsr()

In [40]:
def recom_genres(tag):
    #my_genres = ['인디음악_발라드']

    #my_genre_code = [genre_to_code[g] for g in my_genres]

    #my_genre_ids = [gcode_to_id[c] for c in my_genre_code]

    my_song_ids = tid_sid_dict[tag_to_id[tag]]
    
    # 내가 뽑은 장르 id 만 1인 1차원 희소행렬

    p = np.zeros((len(song_to_id),1))

    for id,count in my_song_ids:
        p[id]=1
        
    # 희소행렬과 dot 연산을 통해 각 태그와 겹치는 장르들의 갯수를 추출
    val = train_tags_A.dot(p).reshape(-1)
    
    cand_gen = train_tags_A_T.dot(val)
    
    # 1차원으로 바꿔서 중복된 장르를 가진 태그 상위 150개 도출
    cand_gen_idx = cand_gen.reshape(-1).argsort()[-150:][::-1]

    # 비슷한 장르 10개 도출
    cand_gen_idx = cand_gen_idx[:10]

    cand_gen_idx
    
    # 원래 노래 id값으로 복원
    result_ids = [id_to_song[i] for i in cand_gen_idx]
    
    
    #result_songs = [(id_to_song[i]) for i in result_ids]
    
    return result_ids

In [50]:
song_ids = recom_genres('백예린')

In [51]:
song_df.loc[song_ids]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
27469,"[GN0401, GN0403]",20160620,Bye bye my blue,2692170,[698776],Bye bye my blue,[GN0400],[백예린],27469
446812,"[GN0401, GN0403]",20151130,FRANK,2653573,[698776],우주를 건너,[GN0400],[백예린],446812
300553,"[GN0401, GN0402]",20190318,Our love is great,10262378,[698776],그건 아마 우리의 잘못은 아닐 거야,[GN0400],[백예린],300553
144663,"[GN0105, GN0101]",20170324,밤편지,10047890,[261143],밤편지,[GN0100],[아이유],144663
342835,"[GN0401, GN0403]",20160620,Bye bye my blue,2692170,[698776],그의 바다,[GN0400],[백예린],342835
419602,"[GN0401, GN0402]",20191210,Every letter I sent you.,10362776,[698776],Square (2017),[GN0400],[백예린],419602
357367,"[GN0401, GN0403]",20160621,비,2692501,[752425],비,[GN0400],[폴킴],357367
505036,"[GN0401, GN0403, GN0402]",20160324,130 mood : TRBL,2674623,[880630],D (half moon) (Feat. 개코),[GN0400],[DEAN],505036
111865,"[GN0401, GN0403]",20151130,FRANK,2653573,[698776],혼자 두지 마,[GN0400],[백예린],111865
601037,[GN0101],20160718,LOVE ANTENNA,2697979,"[760046, 757273]",여기까지,[GN0100],"[권진아, 샘김 (Sam Kim)]",601037
